<a href="https://colab.research.google.com/github/Sauravhuskie/680---assignment-2/blob/main/Thesis_CodeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

Models (CodeBERT): 
1. https://github.com/microsoft/CodeBERT
2. https://github.com/microsoft/CodeBERT/tree/master/CodeBERT/code2nl

Other Sources
2. https://favtutor.com/blogs/print-list-python
3. https://towardsdatascience.com/running-pytorch-on-tpu-a-bag-of-tricks-b6d0130bddd4
4. https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=OApBOAe1fpH_

Diff in Codes:
1. difflib: http://pymotw.com/2/difflib/
2. diff patch match: https://pypi.org/project/diff-match-patch/

TPU Installation
1. https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb#scrollTo=vJZrkoejQhxK

AST
1. https://github.com/pombredanne/python-ast-visualizer/blob/master/astvisualizer.py

2. https://github.com/lensvol/astboom

Similar Products

1. https://github.com/eth-sri/TFix
2. https://github.com/fluidattacks/NIST-SARD-Test-Suites
3. https://github.com/SySeVR/SySeVR

References
1. Code-Bert: https://arxiv.org/pdf/2002.08155.pdf

2. Code Database: https://arxiv.org/ftp/arxiv/papers/2108/2108.04631.pdf

3. SySeVR: https://github.com/SySeVR/SySeVR

Data-Sources

1. https://www.nist.gov/itl/ssd/software-quality-group/static-analysis-tool-exposition-sate-iv

2. SARD:  https://www.nist.gov/itl/ssd/software-quality-group/software-assurance-reference-dataset-sard-manual


Neural Network
1. https://www.atmosera.com/blog/text-classification-with-neural-networks/#:~:text=The%20first%20hidden%20layer%20in,rather%20than%20a%20single%20integer.

2. https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/




## Getting TPU Setup


In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)
  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0a0+git8d365ae
    Uninstalling torch-1.11.0a0+git8d365ae:
      Successfully uninstalled torch-1.11.0a0+git8d365ae


In [3]:
pip uninstall torch torchvision --yes

Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0
Found existing installation: torchvision 0.13.0a0+7be2f55
Uninstalling torchvision-0.13.0a0+7be2f55:
  Successfully uninstalled torchvision-0.13.0a0+7be2f55


In [4]:
!pip install torch==1.11.0
!pip install torchvision

  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires torchvision, which is not installed.
  Using cached torchvision-0.12.0-cp37-cp37m-manylinux1_x86_64.whl (21.0 MB)


In [5]:
VERSION = "1.11"  #@param ["1.11", "nightly", "20220315"]  # or YYYYMMDD format
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION
import os 
os.environ['LD_LIBRARY_PATH']='/usr/local/lib'
!echo $LD_LIBRARY_PATH

!sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
!sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
!sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

!ldconfig
!ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.11 ...
Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0
Found existing installation: torchvision 0.12.0
Uninstalling torchvision-0.12.0:
  Successfully uninstalled torchvision-0.12.0
Copying gs://tpu-pytorch/wheels/colab/torch-1.11-cp37-cp37m-linux_x86_64.whl...
| [1 files][ 98.8 MiB/ 98.8 MiB]                                                
Operation completed over 1 objects/98.8 MiB.                                     
Done updating TPU runtime
Copying gs://tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl...
\ [1 files][145.8 MiB/145.8 MiB]                                                
Operatio

In [6]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [7]:
# Creates a random tensor on xla:1 (a Cloud TPU core)
dev = xm.xla_device()
t1 = torch.ones(3, 3, device = dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


In [8]:
print(dev)

xla:1


In [9]:
# Creating a tensor on the second Cloud TPU core
dev2 = xm.xla_device(n=2, devkind='TPU')
t2 = torch.zeros(3, 3, device = dev2)
print(t2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='xla:2')


In [10]:
print(dev2)

xla:2


In [11]:
# Creating a tensor on the second Cloud TPU core
dev3 = xm.xla_device(n=3, devkind='TPU')
t2 = torch.zeros(3, 3, device = dev3)
print(t2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='xla:3')


In [12]:
print(dev3)

xla:3


## Creating NL - PL Pair

In [13]:
!pip install transformers

In [14]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(dev)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")
nl_tokens

['return', 'Ġmaximum', 'Ġvalue']

In [16]:
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

In [17]:
nl_tokens

['return', 'Ġmaximum', 'Ġvalue']

In [18]:
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
tokens

['<s>',
 'return',
 'Ġmaximum',
 'Ġvalue',
 '</s>',
 'def',
 'Ġmax',
 '(',
 'a',
 ',',
 'b',
 '):',
 'Ġif',
 'Ġa',
 '>',
 'b',
 ':',
 'Ġreturn',
 'Ġa',
 'Ġelse',
 'Ġreturn',
 'Ġb',
 '</s>']

In [19]:
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
tvpairs = zip(tokens,tokens_ids)
for i in tvpairs: 
  print (tuple(i))

('<s>', 0)
('return', 30921)
('Ġmaximum', 4532)
('Ġvalue', 923)
('</s>', 2)
('def', 9232)
('Ġmax', 19220)
('(', 1640)
('a', 102)
(',', 6)
('b', 428)
('):', 3256)
('Ġif', 114)
('Ġa', 10)
('>', 15698)
('b', 428)
(':', 35)
('Ġreturn', 671)
('Ġa', 10)
('Ġelse', 1493)
('Ġreturn', 671)
('Ġb', 741)
('</s>', 2)


In [20]:
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
context_embeddings

tensor([[[-0.1423,  0.3766,  0.0443,  ..., -0.2513, -0.3099,  0.3183],
         [-0.5739,  0.1333,  0.2314,  ..., -0.1240, -0.1219,  0.2033],
         [-0.1579,  0.1335,  0.0291,  ...,  0.2340, -0.8801,  0.6216],
         ...,
         [-0.4042,  0.2284,  0.5241,  ..., -0.2046, -0.2419,  0.7031],
         [-0.3894,  0.4603,  0.4797,  ..., -0.3335, -0.6049,  0.4730],
         [-0.1433,  0.3785,  0.0450,  ..., -0.2527, -0.3121,  0.3207]]],
       grad_fn=<NativeLayerNormBackward0>)

In [21]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline
model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

##Mask Prediction

In [22]:

CODE = "if (x <mask> not None) & (x>1)"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
outputs = fill_mask(CODE)


In [23]:
for i in outputs: print ("missing token:", i['token_str'], " with probability",float (i["score"]) * 100, "%")

missing token: >  with probability 22.588132321834564 %
missing token: <  with probability 17.550398409366608 %
missing token: ==  with probability 14.48403149843216 %
missing token: =  with probability 6.862775236368179 %
missing token: is  with probability 6.072976812720299 %


#Patch Detection or Change in Code

## difflib: Use for texts

In [24]:
text1 = """Saurav Is a good boy"""
text1_lines = text1.splitlines()

text2 = """Saurav is a bad boy """
text2_lines = text2.splitlines()

In [25]:
import difflib
d = difflib.Differ()
diff = d.compare(text1_lines, text2_lines)
print ('\n'.join(diff))

- Saurav Is a good boy
?        ^    ^^^

+ Saurav is a bad boy 
?        ^    ^^     +



## diffUtils: Good for codes


In [26]:
!pip install --upgrade setuptools


In [27]:
!python -m pip install diff-match-patch


In [28]:
Source = "I am the very model of a modern Major-General."
Update ="I am the very model of a cartoon individual.matters?"

In [29]:
from diff_match_patch import diff_match_patch

dmp = diff_match_patch()
patches = dmp.patch_make(Source, Update)
diff = dmp.patch_toText(patches)
diff

'@@ -22,25 +22,31 @@\n f a \n-modern Major-General.\n+cartoon individual.matters?\n'

In [30]:
from diff_match_patch import diff_match_patch

dmp = diff_match_patch()
patches = dmp.patch_fromText(diff)
new_text1, _ = dmp.patch_apply(patches, Source)


In [31]:
print(new_text1)

I am the very model of a cartoon individual.matters?


##patch test

In [32]:
add = """ 
def add(a, b):
  return a + b
"""

sub = """
def add(a,b):
  return a - b
"""

In [33]:
codeB2G = """
package testcases.CWE191_Integer_Underflow.s05; 
import testcasesupport.*; 
  
public class CWE191_Integer_Underflow__short_rand_predec_81_goodB2G extends CWE191_Integer_Underflow__short_rand_predec_81_base 
{ 
    public void action(short data ) throws Throwable 
    { 
  
        /* FIX: Add a check to prevent an underflow from occurring */
        if (data > Short.MIN_VALUE) 
        { 
            short result = (short)(--data); 
            IO.writeLine("result: " + result); 
        } 
        else
        { 
            IO.writeLine("data value is too small to decrement."); 
        } 
  
    } 
} 
"""

In [34]:
codeG2B = """
package testcases.CWE191_Integer_Underflow.s05; 
import testcasesupport.*; 
  
public class CWE191_Integer_Underflow__short_rand_predec_81_goodG2B extends CWE191_Integer_Underflow__short_rand_predec_81_base 
{ 
    public void action(short data ) throws Throwable 
    { 
  
        /* POTENTIAL FLAW: if data == Short.MIN_VALUE, this will overflow */
        short result = (short)(--data); 
  
        IO.writeLine("result: " + result); 
  
    } 
} 
} 
"""

In [35]:
def findPatch(Source, Update):
  from diff_match_patch import diff_match_patch
  tmpPatch = diff_match_patch()
  tempPatches = dmp.patch_make(Source, Update)
  diff = dmp.patch_toText(tempPatches)
  tempPatches = dmp.patch_fromText(diff)
  tempdiff = dmp.patch_toText(tempPatches)
  print(tempdiff)

def applyPatch(code,patch):
  from diff_match_patch import diff_match_patch
  dmp = diff_match_patch()
  new_text1, _ = dmp.patch_apply(patch, code)
  print(new_text1)



In [36]:
#Testing Block for two functions
add2sub = findPatch(sub,add)
applyPatch(add, add2sub)
sub2add = findPatch(add,sub)
applyPatch(sub, sub2add)


@@ -1,8 +1,9 @@
+ 
 %0Adef add
@@ -5,16 +5,17 @@
 f add(a,
+ 
 b):%0A  re
@@ -25,8 +25,8 @@
 n a 
--
++
  b%0A

 
def add(a, b):
  return a + b

@@ -1,9 +1,8 @@
- 
 %0Adef add
@@ -4,17 +4,16 @@
 f add(a,
- 
 b):%0A  re
@@ -23,8 +23,8 @@
 n a 
-+
+-
  b%0A


def add(a,b):
  return a - b



In [37]:
#Testing Block for two functions
add2sub = findPatch(codeG2B,codeB2G)
print(add2sub)


@@ -141,11 +141,11 @@
 good
-G2B
+B2G
  ext
@@ -284,34 +284,85 @@
  /* 
-POTENTIAL FLAW:
+FIX: Add a check to prevent an underflow from occurring */%0A       
  if 
+(
 data 
-==
+%3E
  Sho
@@ -377,32 +377,26 @@
 ALUE
-, this will overflow */%0A
+) %0A        %7B %0A    
     
@@ -430,25 +430,26 @@
 --data); %0A  
-%0A
+  
         IO.w
@@ -486,16 +486,127 @@
  %0A  
+      %7D %0A        else%0A        %7B %0A            IO.writeLine(%22data value is too small to decrement.%22); %0A        %7D %0A  
 %0A    
-%7D %0A
 %7D %0A%7D

None


##Creating Data


In [38]:
import os
import shutil

folder = r"/content/drive/MyDrive/Thesis/CWE-078"
subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
dst = '/content/drive/MyDrive/Thesis/CWE-078'


for sub in subfolders:
    for f in os.listdir(sub):
        src = os.path.join(sub, f)
        dst = folder
        if f.endswith('.cpp'):
          shutil.move(src, dst)



In [56]:
import os
import shutil

folder = r"/content/drive/MyDrive/Thesis/CWE195"
subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
dst = '/content/drive/MyDrive/Thesis/CWE195'


for sub in subfolders:
    for f in os.listdir(sub):
        src = os.path.join(sub, f)
        dst = folder
        print ("src: ",  src , "dst: ",dst)
        if f.endswith('.cpp'):
          shutil.move(src, dst)


src:  /content/drive/MyDrive/Thesis/CWE195/206/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_74a.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/206/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_74b.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/205/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_73a.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/205/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_73b.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/204/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_72b.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/204/CWE195_Signed_to_Unsigned_Conversion_Error__rand_malloc_72a.cpp dst:  /content/drive/MyDrive/Thesis/CWE195
src:  /content/drive/MyDrive/Thesis/CWE195/191/CWE195_Signed_to_Unsigned_Con

##Creating Neural Network